In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20, 6)
import numpy as np

from mmk.utils import audio, show
from mmk.extract.segment import from_recurrence_matrix
from mmk.data import Database, make_root_db

from librosa.util import sync
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances as pwd
import sklearn.cluster as C

In [ ]:
# where you have audio files 
my_music_folder = "not a folder"

# the file with your data :
my_db = "not a db.h5"

if not os.path.exists(my_db):
    make_root_db(my_db, my_music_folder)
    
db = Database(my_db)
piece = db.metadata.iloc[[0]]

X = db.fft.get(piece).T
X.shape

## Segment X

In [ ]:
segments = from_recurrence_matrix(X.T, 
                                 L=6,
                                 k=None,
                                 sym=True,
                                 bandwidth=1.,
                                 thresh=0.2,
                                 min_dur=4)

# aggregate each segment
Sx = sync(X, segments.stop.values, aggregate=np.median, axis=0)

Sx.shape, segments.describe()

## Clustering Methods

In [ ]:
def distance_matrices(X, metric="euclidean", n_neighbors=1, radius=1e-3):
    Dx = pwd(X, X, metric=metric, n_jobs=-1)
    NN = NearestNeighbors(n_neighbors=n_neighbors, radius=radius, metric="precomputed", n_jobs=-1)
    NN.fit(Dx)
    Kx = NN.kneighbors_graph(n_neighbors=n_neighbors, mode='connectivity')
    Rx = NN.radius_neighbors_graph(radius=radius, mode='connectivity')
    return Dx, Kx, Rx


class ArgMax(object):
    def __init__(self):
        self.labels_ = None
        
    def fit(self, X):
        maxes = np.argmax(X, axis=1)
        uniques, self.labels_ = np.unique(maxes, return_inverse=True)
        return self


def cluster(X, Dx=None, n_clusters=128, metric="euclidean", estimator="argmax"):

    estimators = {

        "argmax": ArgMax(),

        "kmeans": C.KMeans(n_clusters=n_clusters,
                           n_init=4,
                           max_iter=200,
                           n_jobs=-1),

         "spectral": C.SpectralClustering(n_clusters=n_clusters, 
                                          affinity="nearest_neighbors",
                                          n_neighbors=32,
                                          assign_labels="discretize",
                                          n_jobs=-1),

        "agglo_ward": C.AgglomerativeClustering(
                                        n_clusters=n_clusters,   
                                        affinity="euclidean",
                                        compute_full_tree='auto',
                                        linkage='ward',
                                        distance_threshold=None,),

        "agglo_single": C.AgglomerativeClustering(
                                    n_clusters=n_clusters,   
                                    affinity="precomputed",
                                    compute_full_tree='auto',
                                    linkage='single',
                                    distance_threshold=None,),

        "agglo_complete": C.AgglomerativeClustering(
                                n_clusters=n_clusters,   
                                affinity="precomputed",
                                compute_full_tree='auto',
                                linkage='complete',
                                distance_threshold=None,)

    }
        
    needs_distances = estimator in {"agglo_single", "agglo_complete"}
    
    if needs_distances:
        if Dx is None:
            Dx, _, _ = distance_matrices(X, metric=metric)
        X_ = Dx
    else:
        X_ = X
        
    cls = estimators[estimator]
    cls.fit(X_)
    
    return cls.labels_

### Ordering functions

In [ ]:
def label_order(labels):
    """return a dictionary where each labels_index (key) is paired 
    with its sorted appearance indices (values)"""
    l_set, indices = np.unique(labels, return_inverse=True)
    rg = np.arange(labels.size)
    return {label: rg[indices == label] for label in l_set}

def replace_by_neighbor(Kx):
    """replace each time_index by one of its neighbor (at random)"""
    order = []
    for i in range(Kx.shape[0]):
        order += [np.random.choice(Kx[i].A.nonzero()[1], 1)[0]]
    return np.r_[order]

def replace_by_label(labels):
    """replace each time_index by an other index having the same label (at random)"""
    l_order = label_order(labels)
    order = []
    for i in range(labels.size):
        candidates = l_order[labels[i]]
        order += [np.random.choice(candidates, 1)[0]]
    return np.r_[order]

def segment_shuffle(segments, sampling_rate=1.):
    """shuffle each segment internally while preserving length and relative order.
    The parameter `sampling_rate` controls how much of each segment is sampled"""
    N = len(segments)
    order = []
    for i in range(N):
        s_i = np.arange(segments.iloc[i, 0].item(), segments.iloc[i, 1].item())
        np.random.shuffle(s_i)
        order += [np.random.choice(s_i, int(len(s_i) * sampling_rate), replace=sampling_rate > 1.)]
    return np.concatenate(order)

## Compute Distances, Neighbors and Clusters

In [ ]:
Dx, Kx, Rx = distance_matrices(X, metric="euclidean", n_neighbors=8, radius=2.)
labels = cluster(X, Dx=Dx, n_clusters=64, estimator="spectral")

# show the distribution of distances
plt.hist(Dx.flat[:], bins=X.shape[0]//4, density=True, alpha=.65)
plt.title("Distances Densities")


plt.figure()
plt.hist(labels, bins=len(set(labels)), color="green", alpha=.65)
plt.title("Cluster's Densities")
None

## Re-Order X by Labels/Neighbors/etc...

In [ ]:
label_o = label_order(labels)
by_neighb = replace_by_neighbor(Kx)
by_label = replace_by_label(labels)
seg_shuff = segment_shuffle(segments)

for k in label_o.keys():
    if len(label_o[k]) <= 1: # this throws error in griffinlim...
        continue
    if len(label_o[k]) > 100: # discard small neighborhoods
        print("label", k)
        audio(X[label_o[k]].T)
    
print("by_neighb")
audio(X[by_neighb].T)

print("by_label")
audio(X[by_label].T)

print("segment_shuffle")
audio(X[seg_shuff].T)
